# Analyse Run #

This is a template notebook to browse the results of a evolution simulation.

Please _Restart & Run All_ to make shure you start with a clean notebook.

## Import required libraries

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from  ipywidgets import widgets 
from ipywidgets import interact, interactive, fixed
from IPython.display import display,HTML,clear_output
import os 
import phievo.AnalysisTools as AT
from  phievo.AnalysisTools import Notebook
notebook = AT.Notebook()
HTML('''<script>code_show=true;function code_toggle() {if (code_show){$('div.input').hide();} else {$('div.input').show();} code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

TypeError: 'module' object is not callable

## Select the Project

In [ ]:
notebook.select_project.display()

## Select Seed

In [ ]:
notebook.select_seed.display()

## Plot observable

In [ ]:
notebook.plot_evolution_observable.display()

## Select Generation

In [ ]:
notebook.select_generation.display()

## PLot Layout

In [ ]:
notebook.plot_layout.display()

## Run Dynamics

In [ ]:
notebook.run_dynamics.display()

## Plot Dynamics

In [ ]:
notebook.plot_dynamics.display()

## Plot Cell Profile

In [ ]:
notebook.plot_cell_profile.display()